# Load Packages and Data

In [1]:
import os
import sys, importlib
from functools import reduce
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import exploratory_analysis as ea

In [2]:
project_path=os.path.dirname(os.path.abspath(''))
superstore_data_path=os.path.join(project_path,"data","superstore.xls")

# Import data
dict_df=ea.import_to_pds(superstore_data_path)

# Print list of loaded tables
print(f"List of tables: {list(dict_df.keys())}")

# Define dataframe variables from loaded excel sheet
df_orders=dict_df["Orders"]
df_people=dict_df["People"]
df_returns=dict_df["Returns"]

List of tables: ['Orders', 'People', 'Returns']


In [3]:
df_orders.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2019-103800,2019-01-03,2019-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2019-112326,2019-01-04,2019-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2019-112326,2019-01-04,2019-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2019-112326,2019-01-04,2019-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2019-141817,2019-01-05,2019-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


# Aggregation test

In [4]:
# We aim to analyze the categories in the data
# --> Per order, add not bought categories

df_sales_category=ea.group_and_fill(
    df=df_orders, 
    cat_nm="Order ID", sub_cat_nm="Category", 
    cols_values=["Sales", "Quantity"], value_for_completion = 0, 
    aggregator="sum", additional_columns="Order Date"
)

# Add additional time columns
df_sales_category=ea.create_time_cols(
    df=df_sales_category,time_col="Order Date",
    to_create=["year", "week", "day_name", "day","month"]
)

df_sales_category.head(5)

,Order ID,Category,Sales,Quantity,Order Date,year,week,day_name,day,month
0,CA-2019-100867,Technology,32.700,3.0,2019-10-19,2019,42,Saturday,292,10
1,CA-2019-100867,Office Supplies,0.000,0.0,2019-10-19,2019,42,Saturday,292,10
2,CA-2019-100867,Furniture,0.000,0.0,2019-10-19,2019,42,Saturday,292,10
3,CA-2019-107153,Technology,0.000,0.0,2019-09-28,2019,39,Saturday,271,9
4,CA-2019-107153,Office Supplies,10.688,2.0,2019-09-28,2019,39,Saturday,271,9


In [9]:
# Median number of ordered quantity in a transaction in year 2022
year=2022
_df_category_sales=df_sales_category[df_sales_category.year==2022].reset_index(drop=True)


median_order_quantity=ea.stat_agg(
    df=_df_category_sales, 
    cat_tables="Category", cat_columns="day_name", cat_rows="month", data_distinguisher=["Order ID"], 
    aggregator="sum", stat_method="mean",
    col_data="Sales", 
    cat_rows_name="Month", cat_columns_name= "Day Name",
    order_cat_columns=['Monday', 'Tuesday', 'Wednesday','Thursday','Friday']
)

median_order_quantity.round(2).fillna("No Data")

Technology                                     Office Supplies  \
Day Name     Monday  Tuesday Wednesday Thursday  Friday          Monday   
Month                                                                     
1             22.07  No Data    336.95    93.18  362.75          477.05   
2             23.95  No Data     46.81   164.14   45.79            8.81   
3             57.40      0.0    701.30   251.16  142.65          231.46   
4             62.79      0.0      9.41   109.91  229.71           41.59   
5             42.44      0.0     44.52    54.02  191.38           66.28   
6            185.78      0.0    120.04    82.98  224.09           45.57   
7            190.18   432.48     92.96   429.90   48.16           55.62   
8            129.14  2183.95    131.03   150.28   54.33          208.32   
9             55.17   583.31     97.86    93.69  150.10          113.99   
10            62.24      0.0    261.39    30.91  149.64          132.94   
11           201.86      0.0    187.95   279.53  284.81           37.48   
12            22.37    20.55     83.25   107.71  130.38          226.08   

                                             Furniture                     \
Day Name  Tuesday Wednesday Thursday  Friday    Monday  Tuesday Wednesday   
Month                                                                       
1         No Data    152.83   499.27  142.26      0.00  No Data    101.96   
2         No Data    154.30   117.62   10.79      0.00  No Data     84.56   
3          308.17     99.91    38.21  104.20    170.55      0.0     39.98   
4           43.09    206.21    21.76  328.61     24.64     0.66     97.46   
5          326.81    140.24    83.83  206.79     97.18   135.31     43.99   
6          288.87    232.48   144.05  104.40     90.36     8.96    214.52   
7           17.05    137.38    78.02  126.12    175.52    173.6    101.27   
8          150.11    292.94   310.54  158.64     48.04     1.39    259.49   
9           55.81    124.14   223.58  128.67     51.03  1832.06     38.38   
10            0.0    253.43   356.85  102.94     68.40    19.98    165.46   
11          57.35    144.42   123.22  109.06    203.66   570.13    130.37   
12           61.7    190.05   133.30  115.93     29.12     62.0     79.56   

                           
Day Name Thursday  Friday  
Month                      
1           46.75   12.93  
2          181.81  192.43  
3           96.57  130.78  
4          179.94  142.07  
5          126.79  110.20  
6          178.74   65.94  
7           84.77  196.97  
8           19.73  118.74  
9          112.21   78.31  
10         105.26   87.83  
11         120.31   95.55  
12         267.80  122.23

In [12]:
# Median number of ordered quantity in a transaction in year 2022
year=2022
_df_category_sales=df_sales_category[df_sales_category.year==2022].reset_index(drop=True)


median_order_quantity=ea.stat_agg(
    df=_df_category_sales, 
    cat_tables="Category", cat_columns="day_name", cat_rows="month", data_distinguisher=["day"], 
    aggregator="nunique", stat_method="mean",
    col_data="Order ID", 
    cat_rows_name="Month", cat_columns_name= "Day Name",
    order_cat_columns=['Monday', 'Tuesday', 'Wednesday','Thursday','Friday']
)

median_order_quantity.round(2).fillna("No Data")

Technology                                    Office Supplies  \
Day Name     Monday  Tuesday Wednesday Thursday Friday          Monday   
Month                                                                    
1              1.67  No Data      3.33     2.25   4.00            1.67   
2              1.00  No Data      2.75     3.00   2.00            1.00   
3              2.75      1.5      4.80     5.20   4.50            2.75   
4              4.67      1.5      3.25     3.75   4.20            4.67   
5              2.60      3.0      5.00     6.00   6.00            2.60   
6              3.00      2.0      5.80     5.25   6.75            3.00   
7              3.67     1.33      3.25     4.25   4.00            3.67   
8              2.80      1.0      4.60     4.25   4.00            2.80   
9              4.50      1.5      8.75     9.60   9.80            4.50   
10             3.00      1.0      8.00     6.25   5.75            3.00   
11             7.00      1.4     10.20    13.00   9.00            7.00   
12             5.00      2.0      8.00     9.40  10.40            5.00   

                                            Furniture                     \
Day Name  Tuesday Wednesday Thursday Friday    Monday  Tuesday Wednesday   
Month                                                                      
1         No Data      3.33     2.25   4.00      1.67  No Data      3.33   
2         No Data      2.75     3.00   2.00      1.00  No Data      2.75   
3             1.5      4.80     5.20   4.50      2.75      1.5      4.80   
4             1.5      3.25     3.75   4.20      4.67      1.5      3.25   
5             3.0      5.00     6.00   6.00      2.60      3.0      5.00   
6             2.0      5.80     5.25   6.75      3.00      2.0      5.80   
7            1.33      3.25     4.25   4.00      3.67     1.33      3.25   
8             1.0      4.60     4.25   4.00      2.80      1.0      4.60   
9             1.5      8.75     9.60   9.80      4.50      1.5      8.75   
10            1.0      8.00     6.25   5.75      3.00      1.0      8.00   
11            1.4     10.20    13.00   9.00      7.00      1.4     10.20   
12            2.0      8.00     9.40  10.40      5.00      2.0      8.00   

                          
Day Name Thursday Friday  
Month                     
1            2.25   4.00  
2            3.00   2.00  
3            5.20   4.50  
4            3.75   4.20  
5            6.00   6.00  
6            5.25   6.75  
7            4.25   4.00  
8            4.25   4.00  
9            9.60   9.80  
10           6.25   5.75  
11          13.00   9.00  
12           9.40  10.40

In [8]:
df_test=ea.agg_cat_stat_in_cells(
    df=_df_category_sales,
    cat_columns="day_name", cat_rows="month", data_distinguisher=["Order ID"],
    cat_in_cell_cols= "Category",
    aggregator="sum", stat_method="median",
    col_data="Quantity",
    cat_rows_name="Month", cat_columns_name= "Day Name",
    order_cat_columns=['Monday', 'Tuesday', 'Wednesday','Thursday','Friday'],
    round_num=2
)

df_test

Day Name,Monday,Tuesday,Wednesday,Thursday,Friday
Month,,,,,
1,Office Supplies: 5,No Data,Office Supplies: 4.5,Office Supplies: 3,"Technology: 2.5, Office Supplies: 5"
2,"Technology: 0.5, Office Supplies: 1.5",No Data,Office Supplies: 5,Office Supplies: 5,Office Supplies: 2
3,Office Supplies: 6,Office Supplies: 7,Office Supplies: 2.5,Office Supplies: 2,Office Supplies: 2.5
4,Office Supplies: 2.5,Office Supplies: 9,Office Supplies: 4,Office Supplies: 2,Office Supplies: 3
5,"Office Supplies: 2, Furniture: 1",Office Supplies: 11,Office Supplies: 3,Office Supplies: 4,Office Supplies: 4
6,Office Supplies: 3,"Office Supplies: 10.5, Furniture: 2.5",Office Supplies: 3,Office Supplies: 4,Office Supplies: 3
7,Office Supplies: 4,"Technology: 2, Office Supplies: 0.5, Furniture...","Office Supplies: 2, Furniture: 3","Technology: 4, Office Supplies: 4","Office Supplies: 5, Furniture: 2"
8,Office Supplies: 2.5,"Technology: 6.5, Office Supplies: 10, Furnitur...",Office Supplies: 5,Office Supplies: 3,Office Supplies: 3
9,Office Supplies: 3,Office Supplies: 6,Office Supplies: 2,Office Supplies: 4,Office Supplies: 4


In [15]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)

In [16]:
ea.agg_cat_stat_in_cells(
    df=_df_category_sales,
    cat_columns="Category", cat_rows="month", data_distinguisher=["Order ID"],
    cat_in_cell_cols= "day_name",
    aggregator="sum", stat_method="median",
    col_data="Quantity",
    cat_rows_name="Month", cat_columns_name= "Day Name",
    round_num=2
)

Day Name,Furniture,Office Supplies,Technology
Month,,,
1,Sunday: 1,"Friday: 5, Wednesday: 4.5, Sunday: 3, Saturday: 4.5, Monday: 5, Thursday: 3",Friday: 2.5
2,No Data,"Friday: 2, Wednesday: 5, Sunday: 3.5, Saturday: 6, Monday: 1.5, Thursday: 5",Monday: 0.5
3,No Data,"Friday: 2.5, Wednesday: 2.5, Sunday: 2.5, Saturday: 3, Monday: 6, Thursday: 2, Tuesday: 7",No Data
4,No Data,"Friday: 3, Wednesday: 4, Sunday: 3, Saturday: 4, Monday: 2.5, Thursday: 2, Tuesday: 9",No Data
5,"Saturday: 2, Monday: 1","Friday: 4, Wednesday: 3, Sunday: 2, Saturday: 3, Monday: 2, Thursday: 4, Tuesday: 11",No Data
6,Tuesday: 2.5,"Friday: 3, Wednesday: 3, Sunday: 3, Saturday: 3, Monday: 3, Thursday: 4, Tuesday: 10.5",No Data
7,"Friday: 2, Wednesday: 3, Tuesday: 1.5","Friday: 5, Wednesday: 2, Saturday: 3, Monday: 4, Thursday: 4, Tuesday: 0.5","Thursday: 4, Tuesday: 2"
8,Tuesday: 1,"Friday: 3, Wednesday: 5, Sunday: 5.5, Saturday: 3, Monday: 2.5, Thursday: 3, Tuesday: 10",Tuesday: 6.5
9,No Data,"Friday: 4, Wednesday: 2, Sunday: 5, Saturday: 2, Monday: 3, Thursday: 4, Tuesday: 6",No Data
